In [1]:
import os
import dotenv
import requests

# bearer_token = os.getenv('paul_sprotte.env')
# bearer = 'Bearer '+ bearer_token
# bearer

## Get Token

In [2]:
# .env
config = dotenv.dotenv_values("paul_sprotte.env")
PASSWORD = config["PASSWORD"]
CLIENT_SECRET = config["CLIENT_SECRET"]

In [3]:
token_url = 'https://accounts.kielregion.addix.io/realms/infoportal/protocol/openid-connect/token'
headers = {
    'Content-Type': 'application/x-www-form-urlencoded'
}

data = {
    'grant_type': 'password',
    'username': 'business.brodmapa@outlook.com', 
    'password': PASSWORD,
    'client_id': 'quantumleap',
    'client_secret': 'fP81XZ5OTt5iRJ7qhyyTCv4eQtpGqc5i'
}

response = requests.post(token_url, headers=headers, data=data)
if response.status_code == 200:
    token_data = response.json()
    access_token = token_data['access_token']
    ACCESS_TOKEN = token_data['access_token']
    print(f"Bearer Token successful requested")
    if access_token:
        dotenv.set_key('paul_sprotte.env', 'ACCESS_TOKEN', access_token)
        print(f"Access Token erfolgreich in die .env-Datei geschrieben.")
        print(access_token)
else:
    print(f"Error: {response.status_code}, {response.text}")

Bearer Token successful requested
Access Token erfolgreich in die .env-Datei geschrieben.
eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJPbWpOY21GRmgwTjV6Wlg1eHg5Zk9mbE0xNFljTmY3WFlieG02OTJtczhBIn0.eyJleHAiOjE3MzM3NDY0OTEsImlhdCI6MTczMzYwMjQ5MSwianRpIjoiN2NjYjU3NTAtMDgwYS00YjNmLTg0M2QtODI2NWNkYWQ1M2RhIiwiaXNzIjoiaHR0cHM6Ly9hY2NvdW50cy5raWVscmVnaW9uLmFkZGl4LmlvL3JlYWxtcy9pbmZvcG9ydGFsIiwiYXVkIjoiYWNjb3VudCIsInN1YiI6IjI0NzQ4NTgxLTU4MGEtNDljZS1iZmY2LTY0YmMwYmExYjY1ZiIsInR5cCI6IkJlYXJlciIsImF6cCI6InF1YW50dW1sZWFwIiwic2Vzc2lvbl9zdGF0ZSI6IjRmZWNkYTkxLWY3YWEtNDM3MS05YjhlLTk3ZTg1NDZhYjZiMiIsImFjciI6IjEiLCJyZWFsbV9hY2Nlc3MiOnsicm9sZXMiOlsiZGVmYXVsdC1yb2xlcy1pbmZvcG9ydGFsIiwib2ZmbGluZV9hY2Nlc3MiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7InF1YW50dW1sZWFwIjp7InJvbGVzIjpbImRhdGEtY29uc3VtZXItcWwiXX0sImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoicm9sZXMgcHJvZmlsZSBlbWFpbCIsInNpZCI6IjRmZWNkYTkxLWY3YWEtNDM3MS05YjhlLT

## Call API

In [4]:
params = {
        "type": "WeatherObserved",
        "fromDate": "2024-10-17T00:00:00.000Z",
        "toDate": "2024-10-18T00:00:00.000Z",
}

header = {
    'NGSILD-Tenant': 'infoportal',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}

collection_responses = []
station_ids = []

url = f"https://apis.kielregion.addix.io/ql/v2/entities"
response = requests.get(url, headers=header, params=params)

if response.status_code == 200:
    response_data = response.json()
    collection_responses.append(response_data)
    print('Data gotten successfully')
else:
    print(f"Error: {response.status_code},{response.text}", end='\r')

Data gotten successfully


In [5]:
entities = []
response_data[0]
for item in response_data:
    if "urn:ngsi-ld:WeatherObserved:OWM:" in item['entityId']:
        entities.append(item['entityId'])

In [6]:
entities

['urn:ngsi-ld:WeatherObserved:OWM:5423951',
 'urn:ngsi-ld:WeatherObserved:OWM:5430967',
 'urn:ngsi-ld:WeatherObserved:OWM:54331015',
 'urn:ngsi-ld:WeatherObserved:OWM:54331021',
 'urn:ngsi-ld:WeatherObserved:OWM:5433971',
 'urn:ngsi-ld:WeatherObserved:OWM:54351009',
 'urn:ngsi-ld:WeatherObserved:OWM:5465941']

In [7]:
# ich habe nur oberflächlich reingeschaut, ber meinem Eindruck nach bracuhe wir nur OWM IDs, 
# Hystreet und sensor.community IDs sind denke ich für andere live messungen gedacht, kommen hier aber auch vor, weil sie Temepraturangabe liefern können (gerne nochmal selbst nachprüfen)
# OWM IDs sind: Rendsburg, Kronshagen, Düsternbrook, Hamdorf, Keelbek, Borgstedt und Schönkirchen. Hamdorf (urn:ngsi-ld:WeatherObserved:OWM:5423951) und Keelbek (urn:ngsi-ld:WeatherObserved:OWM:5465941) sind so weit von ielRegion entfernt, dass wir auf diese Daten verzichten können

# Demo daten für eine Wetterstation (Funktion aus Beispiefunktion bauen)

Demo holt so viele Daten wie möglich für gegebenen Zeitraum. Von der API weden die Daten ab Ende September abgeschnitten, deswegen müssen für komplette Daten zwei abfragen pro ID gemacht werden.
Das Speichern der CSV ist bisher nur Beispielhaft.

Was muss gemacht werden:

* Alle korrekt geholten Daten in CSV anfügen, API wird dich immer wieder rausschmeißen, deswegen muss quesi mit jedem API call stückweise eine CSV weiter geschriben werden und bereits vorhandene Daten geskippt werden
* Mit jeder Anfrage in einem Loop der durch die IDs prüfen ob ID schon in CSV vorhanden ist und ob kompletter Zeitraum vorhanden ist
    * Falls kompletter Zeitraum vorhanden, weiter zur nächsten ID und für aktuelle ID API Request skippen
    * Falls Zeitraum nur teilweise vorhanden, API anfrage von letzter vorhandener Zeitpunkt bis Zeitraum Ende durchführen
    * Falls ID nicht vorhanden, API Request für kompletten Zeitraum durchführen (2024-06-17 bis 2024-12-01)


In [8]:
### Beispiel Funktion für Abrufen einzelner Wetterstation mit ID ###
# Bitte anpassen so, dass sie durch die relevanten IDs iteriert und dabei nach der oben beschroebenen Logik eine CSV mit allen relevanten Wetterdaten schreibt

import requests
import pandas as pd
from datetime import datetime

def get_weather_data(start_date = '2024-06-17T00:00:00.000Z', end_date = '2024-12-01T00:00:00.000Z', entities_ = []):
    weather_list = []
    index_values = []
    """
    Holt Wetterdaten über einen bestimmten Zeitraum.
    
    Args:
        start_date (str): Startdatum im Format 'YYYY-MM-DDT00:00:00'
        end_date (str): Enddatum im Format 'YYYY-MM-DDT23:59:59'
        bearer_token (str): Der Bearer Token für die API-Authentifizierung
        output_file (str): Name der Output-CSV-Datei (Standard: 'weather_data.csv')
    
    Returns:
        pandas.DataFrame: Aggregierte Wetterdaten
    """
    # API Basis-URL und Parameter

    for stationID in entities_:
        base_url = f"https://apis.kielregion.addix.io/ql/v2/entities/{stationID}"
        print(stationID)
    
        params = {
            'type': 'WeatherObserved',
            'fromDate': start_date,
            'toDate': end_date
        }
    
        headers = {
            'NGSILD-Tenant': 'infoportal',
            'Authorization': ACCESS_TOKEN
        }
    
        print("Starte Wetterdatenabruf...")
        
        response = requests.get(base_url, headers=headers, params=params)
        
        if response.status_code == 200:
            data = response.json()
            
            # Daten in DataFrame umwandeln
            weather_data = {}
            index_values.append(data.get('index', []))
            
            for attribute in data.get('attributes', []):
                attr_name = attribute.get('attrName')
                values = attribute.get('values', [])
                weather_data[attr_name] = values
            weather_list.append(weather_data)
        
    return weather_list, index_values

# -------
# Beispielaufruf (brauchen ab 2024-06-17)
start_date = '2024-11-17T00:00:00.000Z'
end_date = '2024-12-01T00:00:00.000Z'

# Funktion aufrufen
weather_data_lis, weather_indecies = get_weather_data(
    start_date = start_date, 
    end_date = end_date, 
    entities_ = entities,
)

urn:ngsi-ld:WeatherObserved:OWM:5423951
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:5430967
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:54331015
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:54331021
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:5433971
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:54351009
Starte Wetterdatenabruf...
urn:ngsi-ld:WeatherObserved:OWM:5465941
Starte Wetterdatenabruf...


In [9]:
# weather_data = pd.DataFrame(weather_data_lis[1])
#df['timestamp'] = pd.to_datetime(weather_indecies)

In [10]:
locations = {}
weather_data_lis[4]['location_centroid']
for i in weather_data_lis:
    locations[i['name'][0]] = i['location_centroid'][0]

In [11]:
locations

{'Hamdorf': '54.226, 9.514',
 'Rendsburg': '54.3024, 9.671',
 'Düsternbrook': '54.3338, 10.1521',
 'Schönkirchen': '54.3262, 10.208',
 'Borgstedt': '54.3317, 9.7092',
 'Kronshagen': '54.346, 10.0893',
 'Keelbek': '54.6545, 9.4101'}

In [12]:
import pandas as pd

# Convert dictionary into a DataFrame
wthr_statn_df = pd.DataFrame(locations.items(), columns=['Place', 'Coordinates'])


wthr_statn_df[['longitude', 'latitude']] = wthr_statn_df['Coordinates'].str.split(',', expand=True).astype(float)
wthr_statn_df = wthr_statn_df.drop(columns=['Coordinates'])
wthr_statn_df.reset_index(drop=True, inplace=True)
wthr_statn_df['wthr_statn_ID'] = range(1, len(wthr_statn_df) + 1)


In [13]:
wthr_statn_df

,Place,longitude,latitude,wthr_statn_ID
0,Hamdorf,54.2260,9.5140,1
1,Rendsburg,54.3024,9.6710,2
2,Düsternbrook,54.3338,10.1521,3
3,Schönkirchen,54.3262,10.2080,4
4,Borgstedt,54.3317,9.7092,5
5,Kronshagen,54.3460,10.0893,6
6,Keelbek,54.6545,9.4101,7


# Group entities by station proximity

In [14]:
entities_df = pd.read_csv("data/station_data.csv")
entities_df

,entityId,station_name,maximum_capacity,longitude,latitude,subarea
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,NaN
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,NaN
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,NaN
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,NaN
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,NaN
...,...,...,...,...,...,...
190,24371,Anleger Reventlou,50,54.333647,10.151714,NaN
191,24370,Alter Markt,25,54.322799,10.139499,NaN
192,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,NaN
193,24368,Umsteiger,22,54.314861,10.131167,NaN


In [ ]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import cdist

# Convert latitude and longitude to radians for Haversine
places_coords = np.radians(entities_df[['latitude', 'longitude']].values)
stations_coords = np.radians(wthr_statn_df[['latitude', 'longitude']].values)

# Haversine formula to calculate distance
def haversine(latlon1, latlon2):
    earth_radius = 6371
    delta_lat = latlon2[:, 0] - latlon1[:, 0][:, None]
    delta_lon = latlon2[:, 1] - latlon1[:, 1][:, None]
    a = np.sin(delta_lat / 2.0) ** 2 + np.cos(latlon1[:, 0][:, None]) * np.cos(latlon2[:, 0]) * np.sin(delta_lon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return earth_radius * c

distances = haversine(places_coords, stations_coords)

# Find the closest station for each place (index of the minimum distance for each row)
closest_station_indices = distances.argmin(axis=1)

# Add the closest station ID and distance to the entities DataFrame
entities_df['closest_station_id'] = wthr_statn_df.iloc[closest_station_indices]['wthr_statn_ID'].values
#entities_df['distance_to_closest_station'] = distances.min(axis=1)

entities_df


,entityId,station_name,maximum_capacity,longitude,latitude,subarea,closest_station_id
0,30434,Felde Gemeindezentrum,18,54.299552,9.933817,NaN,6
1,30207,Sporthalle Saxtorfer Weg,15,54.487005,9.835833,NaN,5
2,30151,Eiche Daenischenhagen,15,54.424663,10.127872,NaN,6
3,30147,Bushaltestelle Eiderbruecke,15,54.289706,10.090042,NaN,6
4,30146,Herzog Friedrich Strasse ZOB,25,54.317114,10.133099,NaN,3
...,...,...,...,...,...,...,...
190,24371,Anleger Reventlou,50,54.333647,10.151714,NaN,3
191,24370,Alter Markt,25,54.322799,10.139499,NaN,3
192,24369,Alte Mu Brunswiker Str,15,54.327583,10.141500,NaN,3
193,24368,Umsteiger,22,54.314861,10.131167,NaN,3


In [16]:
# new df for connection
bike_station_weather = pd.DataFrame()
bike_station_weather['closest_station_id'] = entities_df['closest_station_id']
bike_station_weather['entityId'] = entities_df['entityId']
bike_station_weather

,closest_station_id,entityId
0,6,30434
1,5,30207
2,6,30151
3,6,30147
4,3,30146
...,...,...
190,3,24371
191,3,24370
192,3,24369
193,3,24368


## Export general DFs

In [17]:
bike_station_weather.to_csv('bike_to_weather_station.csv', columns=bike_station_weather.columns, index=False)
wthr_statn_df.to_csv('weather_station.csv', columns=wthr_statn_df.columns, index=False)

## Gather all historical data

In [18]:
import requests
import pandas as pd
from datetime import datetime

# Initialize an empty list to store all data frames
weather_list = []

# Define the output file name
output_file = "historical_weather1.csv"

# 'fromDate' and 'toDate' parameters
params = {
    'type': 'WeatherObserved',
    'fromDate': '2024-12-01T00:00:00.000Z',
    'toDate': '2024-12-07T00:00:00.000Z'
}

headers = {
    'NGSILD-Tenant': 'infoportal',
    'Authorization': ACCESS_TOKEN
}


# Loop through all the entities (stations)
for station in entities:
    base_url = f"https://apis.kielregion.addix.io/ql/v2/entities/{station}"

    print(f"Starte Wetterdatenabruf für Station: {station}")

    # Make the API request
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        print(data)
        # Check if there are attributes to process
        if 'attributes' in data and data['attributes']:
            # Initialize a dictionary to store weather data for the current station
            weather_data = {}
            index_values = data.get('index', [])

            # Process each attribute and add the values to the weather_data dictionary
            for attribute in data['attributes']:
                attr_name = attribute.get('attrName')
                values = attribute.get('values', [])
                weather_data[attr_name] = values

            # Convert the weather data dictionary into a DataFrame
            df = pd.DataFrame(weather_data)
            df['timestamp'] = pd.to_datetime(index_values)

            # Append the DataFrame to the weather_list
            weather_list.append(df)
        else:
            print(f"Keine Wetterdaten für Station {station} gefunden.")
    else:
        print(f"Fehler beim Abrufen der Daten für Station {station}. Status code: {response.status_code}")

# Check if the weather_list is not empty before attempting to concatenate
if weather_list:
    # Concatenate all data frames in the weather_list into one DataFrame
    all_weather_data = pd.concat(weather_list, ignore_index=True)

    # Export the entire data to a CSV file
    all_weather_data.to_csv(output_file, index=False)
    print(f"\nAlle Wetterdaten wurden in {output_file} gespeichert!")
else:
    print("Es wurden keine Wetterdaten abgerufen. Die CSV-Datei wird nicht erstellt.")


Starte Wetterdatenabruf für Station: urn:ngsi-ld:WeatherObserved:OWM:5423951
{'attributes': [{'attrName': 'BatteryVoltage', 'values': [None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, N

In [19]:
historical_weather = pd.read_csv("historical_weather.csv")

In [20]:
historical_weather.name.unique()

array(['Hamdorf', 'Rendsburg', 'Düsternbrook', 'Schönkirchen',
       'Borgstedt', 'Kronshagen', 'Keelbek'], dtype=object)